### Task 4

Теперь проанализируем, влияет ли просмотр ленты новостей на сроки сдачи работ \
 Сроки сдачи соответствующих заданий загружены в таблице **deadlines** \
 Нашей метрикой будет  - разница между датой начала работы над заданием и датой его сдачи

1. Используя библиотеку *sqlite3*, создадим соединение с базой данных

In [1]:
import sqlite3

db_con = sqlite3.connect('..\..\data\checking-logs.sqlite')

2. Выведем информацию о таблице `test`

In [2]:
import pandas as pd

pd.io.sql.read_sql('PRAGMA table_info(test)', db_con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


3. Выведем первые 10 строк таблицы `test`

In [3]:
pd.io.sql.read_sql('SELECT * FROM test LIMIT 10;', db_con)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


4. Найдём и выведем минимальное значение `delta` и имя пользователя с этим минимальным периодом, используя *один* запрос \
Для этого:
    - объединим данные своей таблицы с данными таблицы `deadlines`
    - переведём период времи в часы
    - задание `project1` учитывать не будем, т.к. у него длинный срок сдачи
    - в датафрейме **df_min** сохраним значение `delta` и соответствующий `uid`

In [4]:
df_min =  pd.io.sql.read_sql('SELECT uid, MIN((strftime("%s", first_commit_ts)- deadlines)/3600) as MIN_diff\
                             FROM (SELECT * from test\
                             WHERE labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s"))\
                             LEFT JOIN deadlines ON labname = labs', db_con)
df_min

,uid,MIN_diff
0,user_30,-202


5. Сделаем тоже самое для максимального значения `delta`, используя *один* запрос и название датафрейма - **df_max**

In [5]:
df_max =  pd.io.sql.read_sql('SELECT uid, MAX((strftime("%s", first_commit_ts) - deadlines)/3600) as MAX_diff\
                             FROM (SELECT * from test\
                             WHERE labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s"))\
                             LEFT JOIN deadlines ON labname = labs', db_con)
df_max

,uid,MAX_diff
0,user_25,-2


6. Сделаем тоже самое для среднего значения `delta`, используя *один* запрос, название датафрейма - **df_avg** и в нём будет отсутствовать столбец `uid`

In [6]:
df_avg =  pd.io.sql.read_sql('SELECT AVG((strftime("%s", first_commit_ts) - deadlines)/3600) as AVG_diff\
                             FROM (SELECT * from test\
                             WHERE labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s"))\
                             LEFT JOIN deadlines ON labname = labs', db_con)
df_avg

,AVG_diff
0,-89.125


7. Теперь проверим верно ли, что у учащихся, просматривающих ленту новостей, срок сдачи заданий меньше \
 Рассчитаем для этого коэффициент корреляции между количеством посещений ленты новостей и средним значением `delta` для каждого учащегося
    - С помощью *одного* запрос, создадим таблицу со столбцами: `uid`, `avg_diff`, `pageviews`, где
        - `uid` - это `uid` учащихся из таблицы `test`;
        - `avg_diff` - среднее значение `delta`, т.е. период между датой первого коммита и датой сдачи для каждого задания каждого учащегося;
        - `pageviews` — количество посещений ленты новостей каждым учащимся;
        - задание `project1` учитывать не будем;
        - сохраним данные в датафрейме **views_diff** 
    - Чтоб вычислить коэффициент корреляции между количеством посещений ленты новостей и значением `delta`, воспользуемся методом *corr()* библиотеки **Pandas**

In [7]:
views_diff = pd.io.sql.read_sql('SELECT uid, avg_diff, pageviews FROM (SELECT uid, AVG((strftime("%s", first_commit_ts) - deadlines) / 3600) AS avg_diff\
                                FROM test\
                                LEFT JOIN deadlines\
                                ON test.labname = deadlines.labs\
                                WHERE test.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s")\
                                GROUP BY uid)\
                                LEFT JOIN\
                                (SELECT uid as uid_, COUNT(uid) as pageviews\
                                FROM pageviews\
                                GROUP BY uid)\
                                ON uid = uid_', db_con, index_col="uid")
#views_diff
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


8. Закроим соединение с базой данных

In [8]:
db_con.close()